<a href="https://colab.research.google.com/github/jolinayhsueh/Energy-/blob/main/Supply_forecasting_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MP model




This model is to forecast minerals supply by each country's production and reserves. Time series forecasting method is used for projecting ___ years or supply.

### Data Collection

### Analysis

---
### Coding

Packages:
*   Data preperation: `Numpy`
*   Forecasting
*   Machine learning
*   Data Visualization

Model Sections:
*  [Data preparation](https://colab.research.google.com/drive/1G4WS7w_iM4Ai4h0SlZOCPSuHgArYnYNa#scrollTo=3fY3S6KGaUd4&line=1&uniqifier=1)
*   Model (blah blah)

Data source:
*   [USGS](https://www.sciencebase.gov/catalog/item/63b5f411d34e92aad3caa57f)
*   [World mining data](https://www.world-mining-data.info/?World_Mining_Data___Data_Section)

# Data preparation

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.getcwd()
os.chdir('/content/drive/MyDrive/MP Model')
os.getcwd()
import pandas as pd #imports panda as data frame
import numpy as np #imports numpy as data frame

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Install packages
!pip install pystan prophet
!pip install pre pycaret

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.8/91.8 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 MB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.7/484.7 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 101.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 96.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.5/160.5 kB 20.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
!pip install --upgrade pip setuptools wheel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.5/819.5 kB 24.1 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 67.7.2
    Uninstalling setuptools-67.7.2:
      Successfully uninstalled setuptools-67.7.2
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.


In [4]:
# Importing required libraries
import numpy as np
import pandas as pd, datetime
import seaborn as sns
from statsmodels.tsa.stattools import adfuller
import matplotlib.pyplot as plt
get_ipython().run_line_magic('matplotlib', 'inline')
from time import time
import os
from math import sqrt
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import itertools
import statsmodels.api as sm
from statsmodels.tsa.stattools import acf,pacf
from statsmodels.tsa.arima_model import  ARIMA
from sklearn import model_selection
from sklearn.metrics import mean_squared_error, r2_score
from pandas import DataFrame
import xgboost as xgb
from prophet import Prophet
import warnings
warnings.filterwarnings('ignore')

In [30]:
#Read data

supply_df = pd.read_excel(io="/content/drive/MyDrive/MP Model/Supply Reserves.xlsx")

#Production data
prod = pd.read_excel(io="/content/drive/MyDrive/MP Model/Supply Reserves.xlsx",sheet_name="Production",keep_default_na=False)
proddf = pd.read_excel(io="/content/drive/MyDrive/MP Model/supplydata.xlsx",sheet_name="Production",keep_default_na=False)

In [31]:
# reserves.head()
proddf.dropna(inplace=True)
proddf.head()

,Country,Mineral,year,Prod_m.t
0,Argentina,Li,2017,12360
1,Australia,Li,2017,98650
2,Bolivia,Li,2017,20
3,Brazil,Li,2017,533
4,Canada,Li,2017,66


# Production projection

> Read data -> chang data types -> data validation




In [32]:
# Convert data fromat

proddf['year'] = pd.to_datetime(proddf['year'].astype(str), format='%Y')
proddf['Country'] = proddf['Country'].astype(str)
proddf['Mineral'] = proddf['Mineral'].astype(str)
proddf['Prod_m.t'] = pd.to_numeric(proddf['Prod_m.t'], errors='coerce')


proddf.head()

,Country,Mineral,year,Prod_m.t
0,Argentina,Li,2017-01-01,12360.0
1,Australia,Li,2017-01-01,98650.0
2,Bolivia,Li,2017-01-01,20.0
3,Brazil,Li,2017-01-01,533.0
4,Canada,Li,2017-01-01,66.0


In [33]:
proddf.dtypes

Country             object
Mineral             object
year        datetime64[ns]
Prod_m.t           float64
dtype: object

# Reparameterization

# Forecast Model

In [55]:
#Create forecast library
forecasts = {}

for Country in proddf['Country'].unique():
    for Mineral in proddf['Mineral'].unique():
        country_mineral_df = proddf[(proddf['Country'] == Country) & (proddf['Mineral'] == Mineral)]
        if not country_mineral_df.empty:
            # Prophet requires the columns ds (date) and y (value)
            ts_df = country_mineral_df.rename(columns={'year': 'ds', 'Prod_m.t': 'y'})

            # Create and fit the model
            m = Prophet()
            m.fit(ts_df)

            # Create future dataframe and forecast
            future = m.make_future_dataframe(periods=30, freq='Y')
            forecast = m.predict(future)

            # Store forecast data
            forecasts[(Country, Mineral)] = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]

INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 3.
DEBUG:cmdstanpy:input tempfile: /tmp/tmptn8s_kq0/ze0qjmki.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmptn8s_kq0/hl7bse79.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=50730', 'data', 'file=/tmp/tmptn8s_kq0/ze0qjmki.json', 'init=/tmp/tmptn8s_kq0/hl7bse79.json', 'output', 'file=/tmp/tmptn8s_kq0/prophet_modeldd432ysv/prophet_model-20240124213056.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
21:30:56 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
21:30:56 - cmdstanpy - INFO - Chain [1] done processing
INFO:c

In [54]:
forecasts

{('Argentina',
  'Li'):            ds          yhat    yhat_lower    yhat_upper
 0  2017-01-01  12804.377053  12338.732881  13290.919398
 1  2018-01-01  13295.501599  12861.093786  13750.819038
 2  2019-01-01  13359.930568  12903.956840  13827.178354
 3  2020-01-01  12997.891773  12545.170283  13454.577965
 4  2021-01-01  12873.446965  12428.369365  13326.382404
 5  2021-12-31  10997.916172  10507.943189  11468.812590
 6  2022-12-31   9363.576545   8878.817810   9813.158988
 7  2023-12-31   7309.957407   6855.258843   7779.447466
 8  2024-12-31  12278.870596  11804.887807  12737.839895
 9  2025-12-31  11066.709933  10575.683563  11514.544228
 10 2026-12-31   9432.370306   8935.385132   9897.259042
 11 2027-12-31   7378.751169   6914.062379   7890.435410
 12 2028-12-31  12347.664358  11905.815823  12781.637445
 13 2029-12-31  11135.503695  10615.785885  11605.413499
 14 2030-12-31   9501.164068   9036.665883   9984.232191
 15 2031-12-31   7447.544931   6953.756164   7937.265331
 16 2032

In [73]:
import math

# Define the directory where you want to save the graphs
output_directory = "/content/drive/MyDrive/MP Model/forecast plots"  # Replace with your desired path
os.makedirs(output_directory, exist_ok=True)  # Create the directory if it doesn't exist

# Assuming forecasts is your dictionary
unique_pairs = list(forecasts.keys())
n = len(unique_pairs)

# Calculate rows and columns for subplots
num_cols = math.ceil(math.sqrt(n))
num_rows = math.ceil(n / num_cols)

# Loop through each unique pair and create individual plots
for i, (country, mineral) in enumerate(unique_pairs):
    fig, ax = plt.subplots(figsize=(6, 4))  # Adjust the size as needed

    df = forecasts[(country, mineral)]
    # ax.plot(df['ds'], df['y'], label='Actual')
    ax.plot(df['ds'], df['yhat'], label='Forecast')
    ax.fill_between(df['ds'], df['yhat_lower'], df['yhat_upper'], color='gray', alpha=0.2)
    ax.set_title(f"Forecast for {country} - {mineral}")
    ax.legend()

    # Save the figure
    filename = f"{output_directory}/{country}_{mineral}_forecast.png"
    plt.savefig(filename, format='png', dpi=300)
    # filename = f"{country}_{mineral}_forecast.png"
    # plt.savefig(filename, format='png', dpi=300)

    plt.close(fig)  # Close the figure window

In [74]:
all_forecasts = []

# Iterate over each item in the dictionary
for (country, mineral), forecast_df in forecasts.items():
    # Add country and mineral as columns in each dataframe
    forecast_df['Country'] = country
    forecast_df['Mineral'] = mineral
    all_forecasts.append(forecast_df)

# Concatenate all the dataframes into one
prod_fc_df = pd.concat(all_forecasts)


# Export to Excel
file_name = 'production forecast_data.xlsx'
prod_fc_df.to_excel(file_name, index=False)

print(f"Data exported to {file_name}")

Data exported to production forecast_data.xlsx
